In [1]:
import pandas
csv = pandas.read_csv("crashes.csv")

/tmp/ipykernel_1132347/720344402.py:2: DtypeWarning: Columns (73,80,81,110,115) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pandas.read_csv("crashes.csv")


In [28]:
print(list(csv.columns))

['Date Time', 'Day of Week', 'Object 1', 'Object 2', 'Street Number', 'Street Name', 'Cross Street', 'Near Street', 'Location', 'May involve cyclist', 'May Involve Pedestrian', 'Manner of Collision', 'First Harmful Event Location', 'First Harmful Event', 'Ambient Light', 'Weather Condition 1', 'Weather Condition 2', 'Traffic Control Device Type', 'Traffic Control Device Functionality', 'Road Surface Condition', 'Roadway Junction Type', 'Trafficway Description', 'School Bus Related', 'Work Zone', 'Speed Limit', 'Street or Intersection', 'Intersection Name 1', 'Intersection Direction 1', 'Intersection Name 2', 'Intersection Direction 2', 'Intersection Name 3', 'Intersection Direction 3', 'Street Direction', 'Nearest Intersection Distance', 'Nearest Intersection Direction', 'Landmark Distance', 'Landmark Direction', 'Landmark', 'Mile Marker Direction', 'Non Public Area', 'Road Contributing', 'Damaged Property Type', 'Description of Damaged Property', 'V1 Registration Type', 'v1 State Code

In [64]:
def stats(desc, df):
    print(f"# {desc}")
    print(f"total: {len(df)}")
    p1_injury = df[(~df["P1 Injury"].isnull()) & (df["P1 Injury"] != "NO INJURY") & (df["P1 Injury"] != "NO APPARENT INJURY") &(df["P1 Injury"] != "UNKNOWN") ]
    display(set(p1_injury["P1 Injury"].values))
    p2_injury = df[(~df["P2 Injury"].isnull()) & (df["P2 Injury"] != "NO INJURY") & (df["P2 Injury"] != "NO APPARENT INJURY") &(df["P2 Injury"] != "UNKNOWN")]
    display(set(p2_injury["P2 Injury"].values))
    print("P1 injury: {}".format(len(p1_injury)))
    print("P2 injury: {}".format(len(p2_injury)))
    print("P1 serious injury: {}".format(len(p1_injury[p1_injury["P1 Injury"].isin(["NON FATAL INJURY - INCAPACITATING", "SUSPECTED SERIOUS INJURY"])])))
    print("P2 serious injury: {}".format(len(p2_injury[p2_injury["P2 Injury"].isin(["NON FATAL INJURY - INCAPACITATING", "SUSPECTED SERIOUS INJURY"])])))


In [67]:
# In many cases both may involve cyclist and pedestrian are true... but "Object 1" is a vehicle like PASSENGER CAR, which suggests it's more confusion about who was involved. So we filter it down appropriately.
bike_ped_crash_v1 = csv[(csv["May involve cyclist"] == 1) & (csv["May Involve Pedestrian"] == 1) & (csv["Object 1"].isnull())]
stats("bike + ped", bike_ped_crash_v1)

# bike + ped
total: 23


{'NON FATAL INJURY - INCAPACITATING',
 'NON FATAL INJURY - NON INCAPACITATING',
 'NON FATAL INJURY - POSSIBLE',
 'SUSPECTED MINOR INJURY',
 'SUSPECTED SERIOUS INJURY'}

{'NON FATAL INJURY - NON INCAPACITATING',
 'NON FATAL INJURY - POSSIBLE',
 'SUSPECTED MINOR INJURY'}

P1 injury: 12
P2 injury: 7
P1 serious injury: 2
P2 serious injury: 0


In [68]:
# Another way of identifying bike/ped crashes is via P1 Non Motorist Desc and P2 Non Motorist Desc
bike_ped_crash_v2 = csv[((csv["P1 Non Motorist Desc"] == "PEDESTRIAN") & (csv["P2 Non Motorist Desc"] == "CYCLIST")) | ((csv["P2 Non Motorist Desc"] == "PEDESTRIAN") & (csv["P1 Non Motorist Desc"] == "CYCLIST"))]
# This is undercount, so skipping and using previous metric.

In [69]:
# Car crashes (both single car crash, and two car crash)
cars_only = csv[(csv["May involve cyclist"].isnull()) & (csv["May Involve Pedestrian"].isnull())]
stats("cars only", cars_only)

# cars only
total: 8135


{'DECEASED - NOT CAUSED BY CRASH',
 'FATAL INJURY',
 'NON FATAL INJURY - INCAPACITATING',
 'NON FATAL INJURY - NON INCAPACITATING',
 'NON FATAL INJURY - POSSIBLE',
 'POSSIBLE INJURY',
 'SUSPECTED MINOR INJURY',
 'SUSPECTED SERIOUS INJURY'}

{'FATAL INJURY',
 'NON FATAL INJURY - INCAPACITATING',
 'NON FATAL INJURY - NON INCAPACITATING',
 'NON FATAL INJURY - POSSIBLE',
 'POSSIBLE INJURY',
 'SUSPECTED MINOR INJURY',
 'SUSPECTED SERIOUS INJURY'}

P1 injury: 478
P2 injury: 339
P1 serious injury: 29
P2 serious injury: 20


In [70]:
# Car and ped
cars_ped = csv[(csv["May involve cyclist"].isnull()) & (csv["May Involve Pedestrian"] == 1)]
stats("cars/trucks + ped", cars_ped)


# cars/trucks + ped
total: 500


{'FATAL INJURY',
 'NON FATAL INJURY - INCAPACITATING',
 'NON FATAL INJURY - NON INCAPACITATING',
 'NON FATAL INJURY - POSSIBLE',
 'POSSIBLE INJURY',
 'SUSPECTED MINOR INJURY',
 'SUSPECTED SERIOUS INJURY'}

{'NON FATAL INJURY - INCAPACITATING',
 'NON FATAL INJURY - NON INCAPACITATING',
 'NON FATAL INJURY - POSSIBLE',
 'POSSIBLE INJURY',
 'SUSPECTED MINOR INJURY',
 'SUSPECTED SERIOUS INJURY'}

P1 injury: 86
P2 injury: 196
P1 serious injury: 3
P2 serious injury: 17


In [71]:
# Car and bike
cars_bike = csv[(csv["May involve cyclist"]== 1) & (csv["May Involve Pedestrian"].isnull())]
stats("car/truck + bike", cars_bike)

# car/truck + bike
total: 872


{'NON FATAL INJURY - INCAPACITATING',
 'NON FATAL INJURY - NON INCAPACITATING',
 'NON FATAL INJURY - POSSIBLE',
 'POSSIBLE INJURY',
 'SUSPECTED MINOR INJURY',
 'SUSPECTED SERIOUS INJURY'}

{'FATAL INJURY',
 'NON FATAL INJURY - INCAPACITATING',
 'NON FATAL INJURY - NON INCAPACITATING',
 'NON FATAL INJURY - POSSIBLE',
 'POSSIBLE INJURY',
 'SUSPECTED MINOR INJURY',
 'SUSPECTED SERIOUS INJURY'}

P1 injury: 99
P2 injury: 361
P1 serious injury: 8
P2 serious injury: 14


In [72]:
car_and_non_car_mystery = csv[(csv["May involve cyclist"] == 1) & (csv["May Involve Pedestrian"] == 1) & (~csv["Object 1"].isnull())]
car_and_cyclists_bonus = car_and_non_car_mystery[(csv["P1 Non Motorist Desc"] == "CYCLIST") | (csv["P2 Non Motorist Desc"] == "CYCLIST")]
stats("car and cyclists, bonus", car_and_cyclists_bonus)

car_and_peds_bonus = car_and_non_car_mystery[(csv["P1 Non Motorist Desc"] == "PEDESTRIAN") | (csv["P2 Non Motorist Desc"] == "PEDESTRIAN")]
stats("car and peds, bonus", car_and_peds_bonus)

print("car + ? bonus total", len(car_and_non_car_mystery))

# car and cyclists, bonus
total: 28


/tmp/ipykernel_1132347/2338845205.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  car_and_cyclists_bonus = car_and_non_car_mystery[(csv["P1 Non Motorist Desc"] == "CYCLIST") | (csv["P2 Non Motorist Desc"] == "CYCLIST")]


{'NON FATAL INJURY - NON INCAPACITATING',
 'NON FATAL INJURY - POSSIBLE',
 'SUSPECTED MINOR INJURY'}

{'NON FATAL INJURY - INCAPACITATING',
 'NON FATAL INJURY - NON INCAPACITATING',
 'NON FATAL INJURY - POSSIBLE',
 'SUSPECTED MINOR INJURY',
 'SUSPECTED SERIOUS INJURY'}

P1 injury: 4
P2 injury: 14
P1 serious injury: 0
P2 serious injury: 3
# car and peds, bonus
total: 110


/tmp/ipykernel_1132347/2338845205.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  car_and_peds_bonus = car_and_non_car_mystery[(csv["P1 Non Motorist Desc"] == "PEDESTRIAN") | (csv["P2 Non Motorist Desc"] == "PEDESTRIAN")]


{'NON FATAL INJURY - INCAPACITATING',
 'NON FATAL INJURY - NON INCAPACITATING',
 'NON FATAL INJURY - POSSIBLE',
 'POSSIBLE INJURY',
 'SUSPECTED MINOR INJURY',
 'SUSPECTED SERIOUS INJURY'}

{'NON FATAL INJURY - INCAPACITATING',
 'NON FATAL INJURY - NON INCAPACITATING',
 'NON FATAL INJURY - POSSIBLE',
 'POSSIBLE INJURY',
 'SUSPECTED MINOR INJURY',
 'SUSPECTED SERIOUS INJURY'}

P1 injury: 16
P2 injury: 71
P1 serious injury: 3
P2 serious injury: 10
car + ? bonus total 150


In [73]:
# Ideas: filter out roads not under city control (Alewife Brook Parkway, Memorial Drive)